In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.preprocessing import StandardScaler
import os 
import numpy as np

2024-07-12 15:59:23.452161: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-07-12 15:59:24.577146: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
if tf.config.list_physical_devices('GPU'):
  print("GPU is available")
  print("CuDNN is enabled: True")
else:
  print("GPU is not available")

GPU is available
CuDNN is enabled: True


2024-07-12 15:59:25.442780: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-07-12 15:59:25.506166: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-07-12 15:59:25.506636: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

In [2]:
import pandas as pd
from sklearn.model_selection import KFold

# Örnek DataFrame
data = pd.DataFrame({
    'Millis': [1, 2, 3, 4, 5, 6, 7, 8],
    'ECG': [10, 20, 30, 40, 50, 60, 70, 80],
    'IR': [15, 25, 35, 45, 55, 65, 75, 85],
    'RED': [12, 22, 32, 42, 52, 62, 72, 82],
    'patient': [1, 1, 2, 2, 3, 3, 4, 4],
    'rate_min': [110, 120, 130, 140, 150, 160, 170, 180],
    'rate_max': [210, 220, 230, 240, 250, 260, 270, 280],
    'rate_std': [310, 320, 330, 340, 350, 360, 370, 380],
    'rate_avg': [410, 420, 430, 440, 450, 460, 470, 480],
    'rate_25%': [510, 520, 530, 540, 550, 560, 570, 580],
    'rate_50%': [610, 620, 630, 640, 650, 660, 670, 680],
    'rate_75%': [710, 720, 730, 740, 750, 760, 770, 780]
})

# Hastaları bul ve gruplandır
patients = data['patient'].unique()

# KFold nesnesi oluştur
kf = KFold(n_splits=len(patients) // 2)

# Foldlar üzerinden döngü
for train_index, test_index in kf.split(patients):
    train_patients = patients[train_index]
    test_patients = patients[test_index]

    train_data = data[data['patient'].isin(train_patients)]
    test_data = data[data['patient'].isin(test_patients)]

    print("Train Data")
    print(train_data)
    print("Test Data")
    print(test_data)


Train Data
   Millis  ECG  IR  RED  patient  rate_min  rate_max  rate_std  rate_avg  \
4       5   50  55   52        3       150       250       350       450   
5       6   60  65   62        3       160       260       360       460   
6       7   70  75   72        4       170       270       370       470   
7       8   80  85   82        4       180       280       380       480   

   rate_25%  rate_50%  rate_75%  
4       550       650       750  
5       560       660       760  
6       570       670       770  
7       580       680       780  
Test Data
   Millis  ECG  IR  RED  patient  rate_min  rate_max  rate_std  rate_avg  \
0       1   10  15   12        1       110       210       310       410   
1       2   20  25   22        1       120       220       320       420   
2       3   30  35   32        2       130       230       330       430   
3       4   40  45   42        2       140       240       340       440   

   rate_25%  rate_50%  rate_75%  
0       510  

In [3]:
cwd = os.getcwd()
df = pd.read_csv(cwd + "/main.csv")
df = df.drop('Unnamed: 0', axis=1)


In [4]:
df.head()

,Millis,ECG,IR,RED,patient,datetime,rate_min,rate_max,rate_std,rate_avg,rate_25%,rate_50%,rate_75%,result
0,892,226,28556,28966,1,2023-10-18 09:38:39,0,0,0.0,0.0,0.0,0.0,0.0,1
1,893,226,28554,28959,1,2023-10-18 09:38:39,0,0,0.0,0.0,0.0,0.0,0.0,1
2,893,226,28556,28960,1,2023-10-18 09:38:39,0,0,0.0,0.0,0.0,0.0,0.0,1
3,893,226,28552,28959,1,2023-10-18 09:38:39,0,0,0.0,0.0,0.0,0.0,0.0,1
4,894,226,28551,28954,1,2023-10-18 09:38:39,0,0,0.0,0.0,0.0,0.0,0.0,1


In [5]:
from sklearn.preprocessing import StandardScaler
features = ['Millis', 'ECG', 'IR', 'RED', 'rate_min', 'rate_max',
            'rate_std', 'rate_avg', 'rate_25%', 'rate_50%', 'rate_75%']

scaler = StandardScaler()
df[features] = scaler.fit_transform(df[features])

In [6]:
df = df.drop('datetime', axis=1)


In [7]:
df.columns

Index(['Millis', 'ECG', 'IR', 'RED', 'patient', 'rate_min', 'rate_max',
       'rate_std', 'rate_avg', 'rate_25%', 'rate_50%', 'rate_75%', 'result'],
      dtype='object')

In [8]:
featues = ['patient', 'Millis', 'ECG', 'IR', 'RED', 'rate_min', 'rate_max', 'rate_std', 'rate_avg', 'rate_25%', 'rate_50%', 'rate_75%']
new_df = pd.DataFrame(columns=featues)

In [9]:
grouped = df.groupby('patient')

for name, group in grouped:
    new_row = [name, np.array(group['Millis']),
        np.array(group['ECG']),
        np.array(group['IR']),
        np.array(group['RED']),
        np.array(group['rate_min']),
        np.array(group['rate_max']),
        np.array(group['rate_std']),
        np.array(group['rate_avg']),
        np.array(group['rate_25%']),
        np.array(group['rate_50%']),
        np.array(group['rate_75%'])
    ]
    new_df.loc[len(new_df)] = new_row
    

In [10]:
new_df = new_df.set_index('patient')

In [11]:
new_df

,Millis,ECG,IR,RED,rate_min,rate_max,rate_std,rate_avg,rate_25%,rate_50%,rate_75%
patient,,,,,,,,,,,
1,"[1.3615829951328984, 1.3650514346614286, 1.365...","[0.6103187563610961, 0.6103187563610961, 0.610...","[0.2810035764818336, 0.2804816899821697, 0.281...","[-0.12039222150641705, -0.12416605965368298, -...","[0.353012150214737, 0.353012150214737, 0.35301...","[-0.34699036256971266, -0.34699036256971266, -...","[-0.5006055044779593, -0.5006055044779593, -0....","[7.566185701604071e-05, 7.566185701604071e-05,...","[0.20101720776412163, 0.20101720776412163, 0.2...","[0.0007789585381775566, 0.0007789585381775566,...","[-0.19797306715916266, -0.19797306715916266, -..."
2,"[0.026233776648811418, 0.03317065570587161, 0....","[0.4158561150792519, 0.4158561150792519, 0.415...","[0.7259118174453418, 0.7264337039450058, 0.726...","[-0.5247320229991959, -0.5236537835285484, -0....","[0.353012150214737, 0.353012150214737, 0.35301...","[-0.34699036256971266, -0.34699036256971266, -...","[-0.5006055044779593, -0.5006055044779593, -0....","[7.566185701604071e-05, 7.566185701604071e-05,...","[0.20101720776412163, 0.20101720776412163, 0.2...","[0.0007789585381775566, 0.0007789585381775566,...","[-0.19797306715916266, -0.19797306715916266, -..."
3,"[-1.7045175480877066, -1.6975806690306465, -1....","[4.110646299434292, 3.916183658152447, 3.72172...","[-1.0910360311346627, -1.0915579176343266, -1....","[-0.84982122339939, -0.8508994628700375, -0.85...","[0.353012150214737, 0.353012150214737, 0.35301...","[-0.34699036256971266, -0.34699036256971266, -...","[-0.5006055044779593, -0.5006055044779593, -0....","[7.566185701604071e-05, 7.566185701604071e-05,...","[0.20101720776412163, 0.20101720776412163, 0.2...","[0.0007789585381775566, 0.0007789585381775566,...","[-0.19797306715916266, -0.19797306715916266, -..."
4,"[-0.36223145054655936, -0.35529457148949917, -...","[0.9992440389247844, 0.9992440389247844, 0.999...","[-0.07414018653947704, -0.07414018653947704, -...","[-0.22605968962986325, -0.22605968962986325, -...","[0.353012150214737, 0.353012150214737, 0.35301...","[-0.34699036256971266, -0.34699036256971266, -...","[-0.5006055044779593, -0.5006055044779593, -0....","[7.566185701604071e-05, 7.566185701604071e-05,...","[0.20101720776412163, 0.20101720776412163, 0.2...","[0.0007789585381775566, 0.0007789585381775566,...","[-0.19797306715916266, -0.19797306715916266, -..."
5,"[0.768479835754252, 0.7788851543398423, 0.7823...","[0.4158561150792519, 0.4158561150792519, 0.415...","[0.6844218407220587, 0.6844218407220587, 0.683...","[-0.6994068172440763, -0.7004850567147237, -0....","[0.353012150214737, 0.353012150214737, 0.35301...","[-0.34699036256971266, -0.34699036256971266, -...","[-0.5006055044779593, -0.5006055044779593, -0....","[7.566185701604071e-05, 7.566185701604071e-05,...","[0.20101720776412163, 0.20101720776412163, 0.2...","[0.0007789585381775566, 0.0007789585381775566,...","[-0.19797306715916266, -0.19797306715916266, -..."
6,"[0.8101011100966132, 0.8170379891536733, 0.820...","[1.3881693214884727, 1.3881693214884727, 1.193...","[-2.421324718778044, -2.421585662027876, -2.42...","[-1.1393285212682196, -1.140406760738867, -1.1...","[0.353012150214737, 0.353012150214737, 0.35301...","[-0.34699036256971266, -0.34699036256971266, -...","[-0.5006055044779593, -0.5006055044779593, -0....","[7.566185701604071e-05, 7.566185701604071e-05,...","[0.20101720776412163, 0.20101720776412163, 0.2...","[0.0007789585381775566, 0.0007789585381775566,...","[-0.19797306715916266, -0.19797306715916266, -..."
7,"[-1.1044775096519999, -1.0975406305949398, -1....","[0.6103187563610961, 0.6103187563610961, 0.610...","[-1.8428135339005671, -1.843074477150399, -1.8...","[-1.8256279443352963, -1.8256279443352963, -1....","[0.353012150214737, 0.353012150214737, 0.35301...","[-0.34699036256971266, -0.34699036256971266, -...","[-0.5006055044779593, -0.5006055044779593, -0....","[7.566185701604071e-05, 7.566185701604071e-05,...","[0.20101720776412163, 0.201017207764

In [13]:
pat = pd.read_csv(cwd + "/datasets/hasta.csv")
pat = pat.drop('Unnamed: 0', axis=1)
pat

,patient,result
0,3,LA
1,23,LA DİLATASYONU
2,25,LA DİLATASYONU
3,27,LA DİLATASYONU
4,22,LA DİLATASYONU
...,...,...
84,36,HAFİF MY/TY
85,30,ORTA MY
86,30,İLERİ TY
87,1,HAFİF MY/TY


In [16]:
pat = pat[(pat.result=='LA') | (pat.result=='LA DİLATASYONU')]

In [17]:
new_df = pd.merge(new_df, pat, on='patient')
new_df

,patient,Millis,ECG,IR,RED,rate_min,rate_max,rate_std,rate_avg,rate_25%,rate_50%,rate_75%,result
0,1,"[1.3615829951328984, 1.3650514346614286, 1.365...","[0.6103187563610961, 0.6103187563610961, 0.610...","[0.2810035764818336, 0.2804816899821697, 0.281...","[-0.12039222150641705, -0.12416605965368298, -...","[0.353012150214737, 0.353012150214737, 0.35301...","[-0.34699036256971266, -0.34699036256971266, -...","[-0.5006055044779593, -0.5006055044779593, -0....","[7.566185701604071e-05, 7.566185701604071e-05,...","[0.20101720776412163, 0.20101720776412163, 0.2...","[0.0007789585381775566, 0.0007789585381775566,...","[-0.19797306715916266, -0.19797306715916266, -...",LA DİLATASYONU
1,2,"[0.026233776648811418, 0.03317065570587161, 0....","[0.4158561150792519, 0.4158561150792519, 0.415...","[0.7259118174453418, 0.7264337039450058, 0.726...","[-0.5247320229991959, -0.5236537835285484, -0....","[0.353012150214737, 0.353012150214737, 0.35301...","[-0.34699036256971266, -0.34699036256971266, -...","[-0.5006055044779593, -0.5006055044779593, -0....","[7.566185701604071e-05, 7.566185701604071e-05,...","[0.20101720776412163, 0.20101720776412163, 0.2...","[0.0007789585381775566, 0.0007789585381775566,...","[-0.19797306715916266, -0.19797306715916266, -...",LA
2,3,"[-1.7045175480877066, -1.6975806690306465, -1....","[4.110646299434292, 3.916183658152447, 3.72172...","[-1.0910360311346627, -1.0915579176343266, -1....","[-0.84982122339939, -0.8508994628700375, -0.85...","[0.353012150214737, 0.353012150214737, 0.35301...","[-0.34699036256971266, -0.34699036256971266, -...","[-0.5006055044779593, -0.5006055044779593, -0....","[7.566185701604071e-05, 7.566185701604071e-05,...","[0.20101720776412163, 0.20101720776412163, 0.2...","[0.0007789585381775566, 0.0007789585381775566,...","[-0.19797306715916266, -0.19797306715916266, -...",LA
3,4,"[-0.36223145054655936, -0.35529457148949917, -...","[0.9992440389247844, 0.9992440389247844, 0.999...","[-0.07414018653947704, -0.07414018653947704, -...","[-0.22605968962986325, -0.22605968962986325, -...","[0.353012150214737, 0.353012150214737, 0.35301...","[-0.34699036256971266, -0.34699036256971266, -...","[-0.5006055044779593, -0.5006055044779593, -0....","[7.566185701604071e-05, 7.566185701604071e-05,...","[0.20101720776412163, 0.20101720776412163, 0.2...","[0.0007789585381775566, 0.0007789585381775566,...","[-0.19797306715916266, -0.19797306715916266, -...",LA DİLATASYONU
4,5,"[0.768479835754252, 0.7788851543398423, 0.7823...","[0.4158561150792519, 0.4158561150792519, 0.415...","[0.6844218407220587, 0.6844218407220587, 0.683...","[-0.6994068172440763, -0.7004850567147237, -0....","[0.353012150214737, 0.353012150214737, 0.35301...","[-0.34699036256971266, -0.34699036256971266, -...","[-0.5006055044779593, -0.5006055044779593, -0....","[7.566185701604071e-05, 7.566185701604071e-05,...","[0.20101720776412163, 0.20101720776412163, 0.2...","[0.0007789585381775566, 0.0007789585381775566,...","[-0.19797306715916266, -0.19797306715916266, -...",LA
5,6,"[0.8101011100966132, 0.8170379891536733, 0.820...","[1.3881693214884727, 1.3881693214884727, 1.193...","[-2.421324718778044, -2.421585662027876, -2.42...","[-1.1393285212682196, -1.140406760738867, -1.1...","[0.353012150214737, 0.353012150214737, 0.35301...","[-0.34699036256971266, -0.34699036256971266, -...","[-0.5006055044779593, -0.5006055044779593, -0....","[7.566185701604071e-05, 7.566185701604071e-05,...","[0.20101720776412163, 0.20101720776412163, 0.2...","[0.0007789585381775566, 0.0007789585381775566,...","[-0.19797306715916266, -0.19797306715916266, -...",LA
6,7,"[-1.1044775096519999, -1.0975406305949398, -1....","[0.6103187563610961, 0.6103187563610961, 0.610...","[-1.8428135339005671, -1.843074477150399, -1.8...","[-1.8256279443352963, -1.8256279443352963, -1....","[0.353012150214737, 0.353012150214737, 0.35301...","[-0.34699036256971266, -0.34699036256971266, -...","[-0.5006055044779593, -0.5006055044779593, -0....","[7.566185701604071e-05, 7.5661857016

In [18]:
new_df['result'] = new_df['result'].isin(['LA DİLATASYONU'])

In [30]:
new_df.Millis[0].shape

(14402,)

In [27]:
X = np.array(new_df.drop(columns=['result', 'patient']).values.tolist())
y = new_df['result'].astype(int).values

# Eğitim ve test setlerine bölmek
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Model oluşturma
model = Sequential()
model.add(LSTM(50, input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(Dense(1, activation='sigmoid'))

# Model derleme
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 2 dimensions. The detected shape was (43, 11) + inhomogeneous part.

In [24]:
model = Sequential()
model.add(LSTM(50, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(LSTM(50))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2)


/home/honorsea/miniconda3/envs/tf/lib/python3.10/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type numpy.ndarray).

In [ ]:

# Modeli değerlendirme
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test Loss: {loss}, Test Accuracy: {accuracy}')

In [7]:
scaler = StandardScaler()
scaled_features = scaler.fit_transform(df[['ECG', 'IR', 'RED', 'rate_min', 'rate_max', 'rate_std', 'rate_avg', 'rate_25%', 'rate_50%', 'rate_75%']])

sequence_length = 64
sequences = []
labels = []

for i in range(len(scaled_features) - sequence_length):
    sequences.append(scaled_features[i:i+sequence_length])
    labels.append(df['patient'][i + sequence_length])

len(labels)
import numpy as np
sequences = np.array(sequences)
labels = np.array(labels)

# Veriyi eğitim ve test setlerine böl
X_train, X_test, y_train, y_test = train_test_split(sequences, labels, test_size=0.2, random_state=42)


In [9]:
X_train.shape

(457962, 64, 10)

In [11]:
X_test.shape

(114491, 64, 10)

In [11]:
sequence_length

64

In [8]:

# LSTM modeli oluştur
model = Sequential()
model.add(LSTM(50, return_sequences=True, input_shape=(sequence_length, scaled_features.shape[1])))
model.add(LSTM(50))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=50, batch_size=256, validation_split=0.2)


2024-07-12 14:08:02.166827: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-07-12 14:08:02.167235: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-07-12 14:08:02.167447: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

Epoch 1/50


2024-07-12 14:08:06.236950: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:465] Loaded cuDNN version 8907


1432/1432 ━━━━━━━━━━━━━━━━━━━━ 23s 15ms/step - accuracy: 0.0250 - loss: -530.0878 - val_accuracy: 0.0255 - val_loss: -1641.9545
Epoch 2/50
1432/1432 ━━━━━━━━━━━━━━━━━━━━ 20s 14ms/step - accuracy: 0.0251 - loss: -1998.1735 - val_accuracy: 0.0255 - val_loss: -3067.9729
Epoch 3/50
1432/1432 ━━━━━━━━━━━━━━━━━━━━ 21s 15ms/step - accuracy: 0.0251 - loss: -3424.8064 - val_accuracy: 0.0255 - val_loss: -4490.4160
Epoch 4/50
1432/1432 ━━━━━━━━━━━━━━━━━━━━ 21s 14ms/step - accuracy: 0.0247 - loss: -4852.8179 - val_accuracy: 0.0255 - val_loss: -5912.2002
Epoch 5/50
1432/1432 ━━━━━━━━━━━━━━━━━━━━ 21s 15ms/step - accuracy: 0.0253 - loss: -6279.6426 - val_accuracy: 0.0255 - val_loss: -7333.8931
Epoch 6/50
1432/1432 ━━━━━━━━━━━━━━━━━━━━ 21s 15ms/step - accuracy: 0.0251 - loss: -7705.2603 - val_accuracy: 0.0255 - val_loss: -8756.0186
Epoch 7/50
1432/1432 ━━━━━━━━━━━━━━━━━━━━ 21s 15ms/step - accuracy: 0.0245 - loss: -9121.3984 - val_accuracy: 0.0255 - val_loss: -10178.0967
Epoch 8/50
1432/1432 ━━━━━━━━━━

KeyboardInterrupt: 

In [ ]:

# Modelin performansını değerlendir
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test Accuracy: {accuracy}')


In [34]:
train = df[df['patient'] <= 34]
test = df[df['patient'] > 34]

train_X = train.drop('result', axis=1)
train_y = train.result
test_X = test.drop('result', axis=1)
test_y = test.result

In [35]:
train_X.columns

Index(['Millis', 'ECG', 'IR', 'RED', 'patient', 'rate_min', 'rate_max',
       'rate_std', 'rate_avg', 'rate_25%', 'rate_50%', 'rate_75%'],
      dtype='object')

In [36]:
features = ['Millis', 'ECG', 'IR', 'RED', 'rate_min', 'rate_max',
       'rate_std', 'rate_avg', 'rate_25%', 'rate_50%', 'rate_75%']

for feature in features:
    mean = train_X[feature].mean()
    std = train_X[feature].std()
    
    train_X[feature] = (train_X[feature] - mean) / std
    test_X[feature] = (test_X[feature] - mean) / std


In [37]:
train_X

,Millis,ECG,IR,RED,patient,rate_min,rate_max,rate_std,rate_avg,rate_25%,rate_50%,rate_75%
0,1.362832,0.550931,0.279323,-0.172247,1,0.354197,-0.345987,-0.510328,0.000101,0.199814,0.000931,-0.196114
1,1.366302,0.550931,0.278818,-0.176220,1,0.354197,-0.345987,-0.510328,0.000101,0.199814,0.000931,-0.196114
2,1.366302,0.550931,0.279323,-0.175652,1,0.354197,-0.345987,-0.510328,0.000101,0.199814,0.000931,-0.196114
3,1.366302,0.550931,0.278313,-0.176220,1,0.354197,-0.345987,-0.510328,0.000101,0.199814,0.000931,-0.196114
4,1.369771,0.550931,0.278060,-0.179058,1,0.354197,-0.345987,-0.510328,0.000101,0.199814,0.000931,-0.196114
...,...,...,...,...,...,...,...,...,...,...,...,...
501416,-1.398832,-0.638541,-0.795008,-0.676850,34,0.060520,0.596135,0.389383,0.457574,0.255110,0.454888,0.578997
501417,-1.398832,-0.638541,-0.796523,-0.675715,34,-1.486455,-0.642231,0.597474,-1.250948,-1.396926,-1.155968,-0.854653
501418,-1.398832,-0.638541,-0.796018,-0.675148,34,-1.846904,-2.142967,-0.288654,-2.455087,-2.103472,-2.322827,-2.323688
501419,-1.395362,-0.638541,-0.795766,-0.676850,34,-1.666088,-0.557836,0.762018,-1.317891,-1.509699,-1.226648,-0.864303


In [38]:
test_X

,Millis,ECG,IR,RED,patient,rate_min,rate_max,rate_std,rate_avg,rate_25%,rate_50%,rate_75%
501421,0.124247,0.154441,0.831772,1.155389,35,0.354197,-0.345987,-0.510328,0.000101,0.199814,0.000931,-0.196114
501422,0.131186,-0.043805,0.832277,1.152551,35,0.354197,-0.345987,-0.510328,0.000101,0.199814,0.000931,-0.196114
501423,0.131186,-0.043805,0.831267,1.153119,35,0.354197,-0.345987,-0.510328,0.000101,0.199814,0.000931,-0.196114
501424,0.134655,-0.043805,0.831014,1.151416,35,0.354197,-0.345987,-0.510328,0.000101,0.199814,0.000931,-0.196114
501425,0.138124,-0.043805,0.831014,1.151416,35,0.354197,-0.345987,-0.510328,0.000101,0.199814,0.000931,-0.196114
...,...,...,...,...,...,...,...,...,...,...,...,...
572512,-1.187197,-0.836786,-0.302680,0.326681,41,-0.552243,0.177031,0.582588,0.039765,-0.140998,0.206545,0.253021
572513,-1.149033,-0.836786,-0.302174,0.325546,41,-0.554607,0.884915,1.309880,0.379349,-0.258756,0.466133,0.988918
572514,-1.142094,-0.836786,-0.302680,0.326114,41,0.709918,0.733348,-0.158025,0.816396,0.809163,0.748852,0.622195
572515,-1.135155,-0.836786,-0.303185,0.325546,41,-1.217596,-0.021613,0.727357,-0.849535,-1.053777,-0.764338,-0.555178


In [39]:
model = Sequential()
model.add(LSTM(50, return_sequences=True, input_shape=(12, (train_X.shape[1]-1))))
model.add(LSTM(50))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.fit(train_X, train_y, epochs=10, batch_size=64, validation_split=0.2, verbose=1)


2024-07-12 13:38:14.038151: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-07-12 13:38:14.090690: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-07-12 13:38:14.090998: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

Epoch 1/10


ValueError: Exception encountered when calling Sequential.call().

[1mInvalid input shape for input Tensor("data:0", shape=(None, 12), dtype=float32). Expected shape (None, 12, 11), but input has incompatible shape (None, 12)[0m

Arguments received by Sequential.call():
  • inputs=tf.Tensor(shape=(None, 12), dtype=float32)
  • training=True
  • mask=None

In [ ]:

# Modelin performansını değerlendir
loss, accuracy = model.evaluate(test_X, test_y)
print(f'Test Accuracy: {accuracy}')